# Navier Stokes Equation Reduced Basis Method 
Incompressible timedependet Navier Stokes equation

Find $(u,p):[0,T] \to (H_{0,D}^1)^d \times L^2$, s.t.

\begin{align}
\int_{\Omega} \partial_t u \cdot v + \int_{\Omega} \nu \nabla u \nabla v + u \cdot \nabla u v - \int_{\Omega} \operatorname{div}(v) p &= \int f v  && \forall v \in (H_{0,D}^1)^d, \\ 
- \int_{\Omega} \operatorname{div}(u) q &= 0 && \forall q \in L^2, \\
\quad u(t=0) & = u_0
\end{align}

In [1]:
from netgen import gui
from ngsolve import *
from netgen.geom2d import SplineGeometry
import scipy.sparse as sp
import numpy as np
from math import pi

In [2]:
from netgen.geom2d import SplineGeometry
geo = SplineGeometry()
geo.AddRectangle( (0, 0), (2, 0.41), bcs = ("wall", "outlet", "wall", "inlet"))
geo.AddCircle ( (0.2, 0.2), r=0.05, leftdomain=0, rightdomain=1, bc="cyl")
mesh = Mesh( geo.GenerateMesh(maxh=0.08))
mesh.Curve(3)
Draw(mesh)
# viscosity
nu = 0.001
dt = 0.001

T = 1
dn = int(T/dt)
z = 100  # Number of snapshots

In [3]:
k = 3
V = VectorH1(mesh,order=k, dirichlet="wall|cyl|inlet", dgjumps=True)
Q = H1(mesh,order=k-1)
X = FESpace([V,Q])

In [4]:
gfu_N = GridFunction(X)
gfu = GridFunction(X)
velocity = gfu.components[0]


uin = CoefficientFunction((1.5*4*y*(0.41-y)/(0.41*0.41),0))
gfu.components[0].Set(uin, definedon=mesh.Boundaries("inlet"))

Draw(gfu.components[0],mesh,"gfu",sd=3)

In [5]:
(u,p), (v,q) = X.TnT()

a = BilinearForm(X)
stokes = (nu*InnerProduct(grad(u),grad(v))-div(u)*q-div(v)*p)*dx
a += stokes
a.Assemble()

f = LinearForm(X)
f.Assemble()

inv_stokes = a.mat.Inverse(X.FreeDofs())

res = f.vec.CreateVector()
res.data = f.vec - a.mat*gfu.vec
gfu.vec.data += inv_stokes * res

In [6]:
# matrix for implicit part of IMEX(1) scheme:
mstar = BilinearForm(X)

mstar += InnerProduct(u,v)*dx + dt*stokes

mstar.Assemble()

mh = BilinearForm(X)

mh += InnerProduct(u,v)*dx
mh.Assemble()

inv = mstar.mat.Inverse(X.FreeDofs())


conv = LinearForm(X)
conv += InnerProduct(grad(velocity)*velocity,v)*dx

In [7]:
rows,cols,vals = a.mat.COO()
Ah = sp.csr_matrix((vals,(rows,cols)))
Fh = f.vec
rows,cols,vals = mh.mat.COO()
Mh = sp.csr_matrix((vals,(rows,cols)))

In [8]:
t = 0
tend = 0
len_h = len(gfu.components[0].vec)
erg_h = np.zeros([len_h,int(dn)+1])
conv_h = np.zeros([len_h,int(dn)])
V_POD = np.zeros([len_h,z]) # creat the orthogonal transformationsmatrix for the reducebasis method
line = np.linspace(0,dn-1,z) # stors the number of linear distributed number of iteration from the solution
S_POD = np.zeros([len_h,z])
#line = np.linspace(0,z,z) # stors the number of linear distributed number of iteration from the solution
#line

$
M\bigg(\frac{u^{n+1}-u^n}{k}\bigg) + A u^{n+1} +C u^{n} = 0
$

$
M^*\bigg(u^{n+1}-u^n\bigg) = -k A u^n - k Cu^n
$

$
M^* = M + kA 
$

In [9]:
# implicit Euler/explicit Euler splitting method:
erg_h[:,0] = gfu.components[0].vec
S_POD[:,0] = gfu.components[0].vec
o = 1
e = int(line[o])
for i in range(int(dn)):
    conv.Assemble()
    res.data = a.mat * gfu.vec + conv.vec
    gfu.vec.data -= dt * inv * res
    
    for r in line:
        if int(r) == i: #desided which solution will be orthogonalirsed
            if i > 0:
                S_POD[:,o] = gfu.components[0].vec
                o = o + 1
    erg_h[:,i+1] = gfu.components[0].vec #stores the solution for comparing later
    t = t + dt
    Redraw()

## POD method
A other way to get the transforamtions matrix is the POD method.
gifen $\mathbb{S} = \{u_h^0,u_h^1,...u_h^{S-1}\}$ where $S$ ist the number of snapshots.

$
C = \mathbb{S}^T\mathbb{S}\\
$

$
C \in \mathbb{R}^{S \times S}
$

decompose the solution $u_h$ into the mean part $\bar{u}_h$ and the fluctuation part $\hat{u}_h$

$\bar{u}_h = \frac{1}{S} \sum_{n=0}^{S-1}u_h^n$  $\,\,\,\,\,\, \bar{u}_h \in \mathbb{R}^{Nh \times S-1}$????

and solve the eigenwalueproblem

$
C\Omega = \Omega\Lambda\\
\Lambda = diag[\lambda_0,...,\lambda_{S-1}], \Omega = [\omega^0,...,\omega^{S-1}]
$

$\lambda_i$ is the $i$th eigenvalue and $\omega^i$ is the corresponding normalized $i$th eigenvector

For a integer $r<<S$ given the $r$th POD basis like:

$
\phi_j = \frac{1}{\sqrt{\lambda_j}}\sum_{n=0}^{r-1}\omega^j_nu_h^n, \,\,\,\,\,\,\,\, j = 0,...,r-1
$

Frage:
For $j=0$  and $r = 3$ the expression becomes to:

$
\phi_0 = \frac{1}{\sqrt{\lambda_0}}\bigg[\omega_0^0 \cdot u_h^0 + \omega_1^0 \cdot u_h^1 +\omega_2^0 \cdot u_h^2\bigg]
$

So $\phi \in \mathbb{R}^{N_h\times r}$

To get a orthogonal basis $V = \{\phi_0,...\phi_{r-1}\}$

In [10]:
ubar = np.zeros(len_h)
#splitting the snapshotsolution like in the paper
for i in range(S_POD.shape[1]):
    ubar = ubar + S_POD[:,i]                   
ubar = 1/S_POD.shape[1] * ubar 
for i in range(S_POD.shape[1]):
    S_POD[:,i] = S_POD[:,i] - ubar

In [11]:
print(ubar)

[ 1.24765049e-16  0.00000000e+00  0.00000000e+00 ... -1.05922749e-01
  2.27022742e-02 -3.49115782e-01]


## POD_Method paper

In [ ]:
C = np.transpose(S_POD).dot(S_POD)
lam,omg = np.linalg.eigh(C)

In [ ]:
lam = np.abs(lam)

In [ ]:
r = 6
V_POD = np.zeros([len_h,r])
for i in range(r):
    for j in range(r):
        V_POD[:,i] += omg[j,i]*S_POD[:,j]
    V_POD[:,i] = 1/np.sqrt(lam[i]) * V_POD[:,i]

In [ ]:
print(V_POD[:,0].dot(V_POD[:,0]))
print(V_POD.shape)

## POD_Method RedBasis Book

In [12]:
r = 30
C = np.transpose(S_POD).dot(S_POD)
V_POD = np.zeros([len_h,r])
U,Sig,VH = np.linalg.svd(C)
VH = np.transpose(VH)
for i in range(r):
    V_POD[:,i] = 1/(np.sqrt(Sig[i])) * S_POD.dot(VH[:,i])


In [13]:
V_POD[:,0].dot(V_POD[:,5])

2.8449465006019636e-16

# Offline decomposition

$C_h(\omega,u,v) = \int_K(\nabla \omega \cdot u): v$

$C0_j = C_h(\bar{u}_h,\bar{u}_h,\phi_j)$

$C1_{i,j} = C_h(\bar{u}_h,\phi_i,\phi_j) + C_h(\phi_i,\bar{u}_h,\phi_j)$

$C_{i,j,k} = C_h(\phi_i,\phi_j,\phi_k)$




In [14]:
def Ch(wh,uh,vh):
    return np.linalg.norm(np.inner(np.outer(wh,uh),np.gradient(vh)))
    

In [15]:
def Ch_ngsolve(wh,uh,vh):
    Ch = 0
    u_N = GridFunction(V)
    v_N = GridFunction(V)
    w_N = GridFunction(V)
    w_N.vec.FV().NumPy()[:] = wh
    u_N.vec.FV().NumPy()[:] = uh
    v_N.vec.FV().NumPy()[:] = vh
        
    Ch += Integrate(InnerProduct(Grad(w_N)*u_N,v_N),mesh)
    
    return Ch

In [16]:
C0 = np.zeros(r)
w = ubar
u = w 
for i in range(r):
    v = V_POD[:,i]
    C0[i] = Ch_ngsolve(w,u,v)


In [17]:
C0

array([-5.17823087e-03, -3.62113580e-06,  1.88441280e-03, -8.74070611e-04,
       -1.37511869e-03, -5.23404515e-04,  7.95489017e-05,  1.35739533e-04,
       -3.76307666e-04,  6.63993214e-04, -9.34086583e-04,  3.43807834e-04,
        2.24116829e-04, -1.87080253e-04,  4.57047999e-04, -5.63959872e-05,
        4.24318527e-04,  3.39391588e-05, -1.68931859e-04,  3.89191896e-04,
        1.82716445e-04, -8.78189048e-05,  1.98487985e-04, -3.14123298e-04,
        1.10366026e-04, -1.04358038e-04,  5.59588231e-05, -4.31239016e-04,
        1.83548290e-04,  2.42363979e-04])

In [18]:
C = np.zeros([r,r,r])
for i in range(r):
    w = V_POD[:,i]
    for j in range(r):
        u = V_POD[:,j]
        for k in range(r):
            v = V_POD[:,k]
            C[i,j,k] = Ch_ngsolve(w,u,v)

In [19]:
C1 = np.zeros([r,r])
for i in range(r):
    w = ubar
    u = V_POD[:,i]
    for j in range(r):
        v = V_POD[:,j]
        C1[i,j] = Ch_ngsolve(w,u,v)
        C1[i,j] += Ch_ngsolve(u,w,v)
    

$B_h^{dg}(u,v) = \sum_{T}\big(\int_T \nabla u : \nabla v dx \big) - \int_{\partial T} (\{\{\nabla u \}\}n) \cdot tang(v) ds - \int_{\partial T} (\{\{\nabla v \}\}n) \cdot tang(u) ds + \int_{\partial T} \frac{4k^2}{h}tang([u]) \cdot tang([v] ds$

In [ ]:
#Integrate(dudx, mesh, definedon=mesh.Boundaries('left'))

In [20]:
def Bh_ngsolve(uh,vh):
    Bh = 0
    u_N = GridFunction(V)
    v_N = GridFunction(V)
    u_N.vec.FV().NumPy()[:] = uh
    v_N.vec.FV().NumPy()[:] = vh
    
    Bh += Integrate(InnerProduct(Grad(u_N),Grad(v_N)),mesh)
    
    return Bh

In [21]:
B0 = np.zeros(r)
u = ubar 
for i in range(r):
    v = V_POD[:,i]
    B0[i] = Bh_ngsolve(u,v)

In [22]:
B1 = np.zeros([r,r])
for i in range(r):
    u = V_POD[:,i]
    for j in range(r):
        v = V_POD[:,j]
        B1[i,j] = Bh_ngsolve(u,v)

In [23]:
B1

array([[ 5.77709474e-02,  1.45455852e-04,  1.06336867e-03,
         6.54984811e-03,  7.90212118e-03,  4.55334510e-03,
         2.20575162e-03,  1.29306161e-03, -8.33087282e-04,
         4.14816423e-03,  7.29062511e-04, -4.10371676e-04,
         1.59560396e-03,  1.54233478e-03,  3.52218209e-04,
         2.06185216e-03, -1.65731534e-03, -6.40013530e-04,
         2.82618213e-03, -2.88083409e-03,  2.54947070e-03,
         7.95014917e-04,  3.39699354e-04,  3.53874880e-03,
        -1.22522863e-03, -7.37285622e-04,  2.06592374e-03,
         2.42557968e-03, -2.89077818e-03, -1.04522820e-03],
       [ 1.45455852e-04,  7.06821684e-02, -2.16238834e-03,
        -3.93495653e-03, -1.51769671e-03,  3.13936793e-03,
         4.21174962e-03, -1.37113530e-02, -7.30372584e-04,
         3.49919929e-03,  1.40177100e-03,  4.24044973e-03,
        -2.01210973e-03, -2.18177355e-07, -3.06823977e-03,
        -1.95282472e-03,  4.41111040e-03, -7.56057661e-04,
         4.52142147e-03,  3.17916435e-03, -3.69680473e-

In [24]:
def Mh_ngsolve(uh,vh):
    Mh = 0
    u_N = GridFunction(V)
    v_N = GridFunction(V)
    u_N.vec.FV().NumPy()[:] = uh
    v_N.vec.FV().NumPy()[:] = vh
    
    Mh += Integrate(InnerProduct(u_N,v_N),mesh)
    
    return Mh

In [25]:
M1 = np.zeros([r,r])
for i in range(r):
    u = V_POD[:,i]
    for j in range(r):
        v = V_POD[:,j]
        M1[i,j] = Mh_ngsolve(u,v)

In [26]:
M1

array([[ 5.45380439e-05,  6.36868456e-07,  7.37644385e-07,
         1.08634969e-05,  2.35592866e-05,  5.29746757e-06,
         2.65830427e-06, -1.87437688e-06, -7.02494455e-07,
         1.89962517e-06,  1.08310322e-06, -1.75717353e-06,
         1.79927793e-06,  1.17127438e-06, -4.22676525e-07,
         2.01998967e-06, -1.91748953e-06, -6.41630151e-07,
         2.68573299e-06, -3.14336341e-06,  1.32896380e-06,
         7.03074638e-07, -4.24787891e-07,  3.55345666e-06,
        -3.26917965e-07,  2.64507859e-08,  1.95818099e-06,
         3.47924615e-06, -3.32095439e-06, -9.63711033e-07],
       [ 6.36868456e-07,  7.88263559e-05, -4.13899261e-06,
        -6.66881359e-06, -6.07427669e-07,  5.51914260e-06,
         9.43165316e-06, -2.40861511e-05, -3.75462304e-06,
         4.76967511e-06,  5.97012035e-06,  9.14788127e-06,
        -3.06686686e-06,  6.96950455e-07, -2.53509761e-06,
        -2.67198179e-06,  7.94193690e-06, -1.20637987e-06,
         5.51650123e-06,  6.04035206e-06,  2.91930133e-

# RedBasis NavierStokes

In [27]:
gfu_N = GridFunction(V)
gfu = GridFunction(V)
gfu_N.vec.FV().NumPy()[:] = erg_h[:,0]
gfu.vec.FV().NumPy()[:] = erg_h[:,0]

uN = np.array(gfu_N.vec).dot(V_POD)
velocity_N = gfu_N.components[0]
Draw(gfu_N.components[0],mesh,"gfu_N",sd=3)
Draw(gfu_N.components[0]-gfu.components[0],mesh,"error",sd=3)
err = res.CreateVector()

In [28]:
dt = 0.00001
dn = 1/dt
Residium = []
time = []
erg_N = np.zeros([len(gfu_N.vec),int(dn)+1])
erg_N[:,0] = gfu_N.vec

I = np.eye(r)
MN_star = M1/dt + 0.5*dt*nu*B1
inv_N = np.linalg.inv(MN_star)
for i in range(int(dn)):
    #err.vec = sqrt (Integrate ((gfu.components[0]-gfu_N.components[0])*(gfu.components[0]-gfu_N.components[0]),mesh))
    
    res_N = C0 + C1.dot(uN) + uN.dot(C.dot(uN)) + nu*B0 + 0.5*nu*B1.dot(uN)
    uN -= dt * inv_N.dot(res_N)
    #erg_N[:,i+1] = (uN).dot(np.transpose(V_POD))
    
    
    #gfu.vec.FV().NumPy()[:] = erg_h[:,i+1]
    gfu_N.vec.FV().NumPy()[:] = (uN).dot(np.transpose(V_POD))
    #Residium.append(err.vec)
    #time.append(dt*i)
    Redraw()

In [ ]:
Residium